In [ ]:
import pandas as pd
import os
import json
import numpy as np
import string

In [ ]:
businesses = pd.read_json('Data/yelp_academic_dataset_business.json', lines = True)
businesses = businesses.drop(['address', 'postal_code', 'longitude', 'latitude', 'is_open'], axis = 1)
businesses.columns = ['business_id', *['business_' + col for col in businesses.columns[1:]]]

check = pd.read_json('Data/yelp_academic_dataset_checkin.json', lines = True)
check.columns = ['business_id', 'checkin_date']

businesses = businesses.join(check.set_index('business_id'), on = 'business_id', how = 'left')

del(check)

In [ ]:
def categorize_tags(term):
    
    category = term
    
    if('auto' in term.lower()): category = 'Automotive'
    elif('restaurant' in term.lower()): category = 'Restuarant'
    elif('food' in term.lower()): category = 'Cafe'
    elif('spa' in term.lower()): category = 'Spa'
    elif('shop' in term.lower()): category = 'Shopping'
    elif('entertainment') in term.lower(): category = 'Arts & Entertainment'
    elif('hotel') in term.lower(): category = 'Hotel'
    elif('pet') in term.lower(): category = 'Pet Services'
    elif('active life') in term.lower(): category = 'Active Life'
    elif('nightlife') in term.lower(): category = 'Nightlife'
    elif('medical') in term.lower(): category = 'Medical'
    elif('home services') in term.lower(): category = 'Home Services'
    elif('education') in term.lower(): category = 'Education'
    elif('local') in term.lower(): category = 'Local Services'
    elif('professional') in term.lower(): category = 'Professional Services'
    elif('financial') in term.lower(): category = 'Financial Services'
    elif('public services') in term.lower(): category = 'Public Services'
    elif('event planning') in term.lower(): category = 'Event Planning'
    elif('religious') in term.lower(): category = 'Religious'
    else: category = 'Unspecified'
    
    return category

In [ ]:
def expand_checkins(dates):
    
    if(dates == 'nan'):
        return 0, 0, 0, 0, 0
    
    hours = []

    num_morn = 0
    num_aft = 0
    num_eve = 0
    num_late = 0

    for j in range(len(dates.split(','))):
        hours.append(int(dates.split(',')[j].strip()[11:13]))

    for k in range(len(hours)):
        if((hours[k] >= 4) & (hours[k] < 12)):
            num_morn += 1
        elif((hours[k] >= 12) & (hours[k] < 18)):
            num_aft += 1
        elif((hours[k] >= 18) & (hours[k] < 23)):
            num_eve += 1
        else:
            num_late += 1

    num_tot = len(range(len(hours)))
    
    return num_morn, num_aft, num_eve, num_late, num_tot

In [ ]:
def days_open(hours):

    hours_trf = json.loads(hours.replace("'", '"'))

    days_open = list(hours_trf.keys())
    num_days_open = len(days_open)
    
    return days_open, num_days_open

businesses['days_open'], businesses['num_days_open'] = businesses['business_hours'].map(lambda x: days_open(x))

In [ ]:
businesses = businesses.rename(columns = {'business_categories': 'business_tags'})

In [ ]:
businesses['business_category'] = businesses['business_tags'].astype(str).map(lambda x: categorize_tags(x))

In [ ]:
checkins_by_TOD = businesses['checkin_date'].astype(str).map(lambda x: expand_checkins(x))
checkins_by_TOD_df = pd.DataFrame(checkins_by_TOD.tolist(), index = checkins_by_TOD.index, columns = 
                                  ['checkins_morn', 'checkins_aft', 'checkins_eve', 'checkins_late', 'checkins_TOTAL'])

businesses = pd.concat([businesses.reset_index(drop = True), checkins_by_TOD_df], axis = 1)

In [ ]:
business_attributes = []

for i in range(len(businesses['business_attributes'])):
    if(str(businesses['business_attributes'][i]) == 'nan'):
        break

    business_attributes.append(str(businesses['business_attributes'][i]).split(':')[0])

business_attributes = list(set(business_attributes))

for i in range(len(business_attributes)):
    business_attributes[i] = business_attributes[i].translate(str.maketrans('', '', string.punctuation))

In [ ]:
business_attributes_df = np.zeros(shape = (len(businesses), len(business_attributes)))
business_attributes_df = pd.DataFrame(business_attributes_df, columns = business_attributes)
businesses = pd.concat([businesses.reset_index(drop = True), business_attributes_df], axis = 1)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

for attr in business_attributes[30:]:
    for bus in range(len(businesses)):
        try:
            businesses_detailed[attr][bus] = businesses_detailed['business_attributes'][bus].get(attr, 0)
        except AttributeError:
            businesses_detailed[attr][bus] = 0

In [ ]:
for attr in businesses_detailed.columns[15:].values:
    for i in range(len(businesses_detailed)):
        if(businesses_detailed[attr][i] == 0):
            businesses_detailed[attr][i] = 'Unspecified'

In [ ]:
for bus in range(len(businesses)):
    try:
        businesses_detailed['BusinessParking'][bus] = businesses_detailed['business_attributes'][bus].get('BusinessParking', 'Unspecified')
    except AttributeError:
        businesses_detailed['BusinessParking'][bus] = 'Unspecified'

def is_BP_true(term):
    if('True' in term):
        return 'True'
    elif('Unspecified' in term):
        return 'Unspecified'
    else:
        return 'False'
    
BusinessParking = [is_BP_true(term) for term in businesses_detailed['BusinessParking']]

businesses_detailed['BusinessParking'] = BusinessParking

In [ ]:
businesses_detailed['Alcohol'] = businesses_detailed['Alcohol'].replace("'none'", 'False')
businesses_detailed['Alcohol'] = businesses_detailed['Alcohol'].replace("none", 'False')
businesses_detailed['Alcohol'] = businesses_detailed['Alcohol'].replace("'beer_and_wine'", 'Beer and Wine Only')
businesses_detailed['Alcohol'] = businesses_detailed['Alcohol'].replace("beer_and_wine", 'Beer and Wine Only')
businesses_detailed['Alcohol'] = businesses_detailed['Alcohol'].replace("'full_bar'", 'Full Bar')
businesses_detailed['Alcohol'] = businesses_detailed['Alcohol'].replace("full_bar", 'Full Bar')

In [ ]:
businesses_detailed['WiFi'] = businesses_detailed['WiFi'].replace("'free'", 'True')
businesses_detailed['WiFi'] = businesses_detailed['WiFi'].replace("free", 'True')
businesses_detailed['WiFi'] = businesses_detailed['WiFi'].replace("'no'", 'False')
businesses_detailed['WiFi'] = businesses_detailed['WiFi'].replace("no", 'False')
businesses_detailed['WiFi'] = businesses_detailed['WiFi'].replace("'paid'", 'Paid Only')
businesses_detailed['WiFi'] = businesses_detailed['WiFi'].replace("paid", 'Paid Only')

In [ ]:
businesses_detailed['RestaurantsPriceRange2'] = businesses_detailed['RestaurantsPriceRange2'].replace("1", '$')
businesses_detailed['RestaurantsPriceRange2'] = businesses_detailed['RestaurantsPriceRange2'].replace(1.0, '$')
businesses_detailed['RestaurantsPriceRange2'] = businesses_detailed['RestaurantsPriceRange2'].replace("2", '$$')
businesses_detailed['RestaurantsPriceRange2'] = businesses_detailed['RestaurantsPriceRange2'].replace(2.0, '$$')
businesses_detailed['RestaurantsPriceRange2'] = businesses_detailed['RestaurantsPriceRange2'].replace("3", '$$$')
businesses_detailed['RestaurantsPriceRange2'] = businesses_detailed['RestaurantsPriceRange2'].replace(3.0, '$$$')
businesses_detailed['RestaurantsPriceRange2'] = businesses_detailed['RestaurantsPriceRange2'].replace("4", '$$$$')
businesses_detailed['RestaurantsPriceRange2'] = businesses_detailed['RestaurantsPriceRange2'].replace(4.0, '$$$$')

In [ ]:
businesses_detailed['Smoking'] = businesses_detailed['Smoking'].replace("'no'", 'False')
businesses_detailed['Smoking'] = businesses_detailed['Smoking'].replace("'yes'", 'True')
businesses_detailed['Smoking'] = businesses_detailed['Smoking'].replace("'outdoor'", 'Outdoors Only')
businesses_detailed['Smoking'] = businesses_detailed['Smoking'].replace("no", 'False')
businesses_detailed['Smoking'] = businesses_detailed['Smoking'].replace("yes", 'True')
businesses_detailed['Smoking'] = businesses_detailed['Smoking'].replace("outdoor", 'Outdoors Only')

In [ ]:
businesses_detailed['NoiseLevel'] = businesses_detailed['NoiseLevel'].replace("'average'", 'Average')
businesses_detailed['NoiseLevel'] = businesses_detailed['NoiseLevel'].replace("average", 'Average')
businesses_detailed['NoiseLevel'] = businesses_detailed['NoiseLevel'].replace("'quiet'", 'Quiet')
businesses_detailed['NoiseLevel'] = businesses_detailed['NoiseLevel'].replace("quiet", 'Quiet')
businesses_detailed['NoiseLevel'] = businesses_detailed['NoiseLevel'].replace("'loud'", 'Loud')
businesses_detailed['NoiseLevel'] = businesses_detailed['NoiseLevel'].replace("loud", 'Loud')
businesses_detailed['NoiseLevel'] = businesses_detailed['NoiseLevel'].replace("'very_loud'", 'Very Loud')
businesses_detailed['NoiseLevel'] = businesses_detailed['NoiseLevel'].replace("very_loud", 'Very Loud')

In [ ]:
businesses_detailed['RestaurantsAttire'] = businesses_detailed['RestaurantsAttire'].replace("'casual'", 'Casual')
businesses_detailed['RestaurantsAttire'] = businesses_detailed['RestaurantsAttire'].replace("casual", 'Casual')
businesses_detailed['RestaurantsAttire'] = businesses_detailed['RestaurantsAttire'].replace("'dressy'", 'Formal')
businesses_detailed['RestaurantsAttire'] = businesses_detailed['RestaurantsAttire'].replace("dressy", 'Formal')
businesses_detailed['RestaurantsAttire'] = businesses_detailed['RestaurantsAttire'].replace("formal", 'Formal')
businesses_detailed['RestaurantsAttire'] = businesses_detailed['RestaurantsAttire'].replace("'formal'", 'Formal')

In [ ]:
businesses_detailed = businesses_detailed.drop(['BestNights', 'Ambience'], axis = 1)

In [ ]:
businesses_detailed['BusinessParking'][9]

In [ ]:
businesses_detailed['business_attributes'][0]['BusinessParking']

In [ ]:
def is_BP_true(term):
    if('BusinessParking' in businesses_detailedterm):
        if 'True' in businesses_detailed['business_attributes'][['BusinessParking']
    else:
        return term
    
businesses_detailed['BusinessParking'] = [is_true(term) for term in businesses_detailed['business_attributes']]

In [ ]:
businesses_detailed = businesses_detailed.drop('business_attributes')

In [ ]:
businesses_detailed.to_csv('businesses_transformed.csv', index = False)